In [1]:
# Path: tests/peak_detect.ipynb
import os
# find root directory
root = os.path.dirname(os.path.abspath('./'))
# append root directory to sys.path
import sys
sys.path.append(root)

# import module
import csv
import numpy as np
import pandas as pd
# import peakdetect
from src.PeakDetect.PeakDetect import PeakDetect
# bokeh plot settings
import pandas_bokeh
from bokeh.io import output_notebook, show, save
from bokeh.models import ColumnDataSource, HoverTool, Scatter
from bokeh.plotting import figure
from bokeh.sampledata.autompg import autompg_clean
from bokeh.transform import factor_cmap
from bokeh.layouts import gridplot
output_notebook()

Loading BokehJS ...

In [2]:
# read data from csv file
data_dir = os.path.join(root, 'data/')
from tools.get_files import GetFiles
file_finder = GetFiles(data_dir, 'csv')
file_list = file_finder.get_files()

100%|██████████| 10/10 [00:00<00:00, 71697.50it/s]


In [3]:
# read csv files
def read_csv(file):
    df = pd.read_csv(file, )
    df.dropna()
    filename = os.path.basename(os.path.splitext(file)[0])
    return df, filename

In [4]:
data_list, filename_list = [], []
for file in file_list:
    data, filename = read_csv(file)
    data = data[['x','y']]
    data_list.append(data)
    filename_list.append(filename)

data = data_list[0]
filename = filename_list[0]

In [5]:
data.plot_bokeh(kind='line', x='x', y='y', title=filename)

Figure(id='1003', ...)

In [6]:
detector = PeakDetect(data, 432, 0.5, 0.1, 5, denoise=False, need_fit=False, filter_name='savgol_filter', peak_type='gaussian', peak_num=None)

In [7]:
peak_index, valley_index, main_peak = detector.find_peaks()

[findpeaks] >Finding peaks in 1d-vector using [topology] method..
[findpeaks] >Interpolating 1d-vector by factor 10
[findpeaks] >Detect peaks using topology method with limit at 5.


In [8]:
peak_index, valley_index, main_peak, correlation_factor = detector.peak_detect()

[findpeaks] >Finding peaks in 1d-vector using [topology] method..
[findpeaks] >Interpolating 1d-vector by factor 10
[findpeaks] >Detect peaks using topology method with limit at 5.


In [9]:
print("peak_index: ", peak_index)
print("valley_index: ", valley_index)
print("main_peak: ", main_peak)
print("correlation_factor: ", correlation_factor)

peak_index:  [[410, 429, 441]]
valley_index:  [423, 440]
main_peak:  [435.3559875488281, 99.99999237060548, 410]
correlation_factor:  0.9922913945258379


In [10]:
width_result = detector.main_peak_width_detect()

[findpeaks] >Finding peaks in 1d-vector using [topology] method..
[findpeaks] >Interpolating 1d-vector by factor 10
[findpeaks] >Detect peaks using topology method with limit at 5.


In [11]:
width_result

0.7104840301014773

In [13]:
test = np.array([[ 1.04364806, 39.78261348],
       [ 1.73212699,  5.92853279],
       [ 0.59810331, 41.64359442]])
result = np.argmax(test[:,1])
width_result = test[result, 0]
width_result

0.59810331

In [14]:
data_np = data.to_numpy()
data_np

array([[4.16997009e+02, 5.00752591e-03],
       [4.17040985e+02, 1.03488881e-02],
       [4.17085999e+02, 1.20514464e-02],
       ...,
       [4.46906006e+02, 1.03822723e-02],
       [4.46950989e+02, 1.03822723e-02],
       [4.46994995e+02, 1.03822723e-02]])

In [9]:
import findpeaks.findpeaks as findpeaks
fp = findpeaks(method='topology', interpolate=10, lookahead=100, limit=5)
try:
    peak_results = fp.fit(data['y'])
except:
    raise Exception('findpeaks failed, please check the data')

[findpeaks] >Finding peaks in 1d-vector using [topology] method..
[findpeaks] >Interpolating 1d-vector by factor 10
[findpeaks] >Detect peaks using topology method with limit at 5.


In [10]:
def peak_process(peaks:dict, threshold:float, data:pd.DataFrame):
        df = peaks['df']
        df['x'] = data['x']
        peak_index= df[lambda df: df['peak'] == True]
        highest_peak_value = df.loc[peak_index.index.tolist()]['y'].max()
        highest_peak_index = df.loc[peak_index.index.tolist()]['y'].idxmax()
        highest_peak_x = df['x'][highest_peak_index]
        highest_peak = [highest_peak_x, highest_peak_value, highest_peak_index]
        peak_index_list = peak_index.loc[peak_index.y.gt(threshold * highest_peak_value)].index.tolist()
        valley_index = df[lambda df: df['valley'] == True]
        valley_index_list = valley_index.loc[valley_index.y.gt(threshold * highest_peak_value)].index.tolist()
        return peak_index_list, valley_index_list, highest_peak

In [12]:
peak_index, valley_index, highest_peak = peak_process(peak_results,  0.1, data)
print("peak_index: ", peak_index)
print("valley_index: ", valley_index)
print("highest_peak: ", highest_peak)

peak_index:  [410, 429, 441]
valley_index:  [423, 440]
highest_peak:  [435.3559875488281, 99.99999237060548, 410]
